# TASK1: load the toxic dataset and append all comments to one long text 

In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('/home/ubuntu/Downloads/train.csv')
total_train_sent_list = list(train_data['comment_text'])

In [3]:
long_text = ''
for line in total_train_sent_list:
    long_text += line
    long_text += ' '

# Task2: build vocabulary 

In [4]:
#we need that every characted will have an index 
long_text = long_text[:10000]
character_list = [character for character in long_text]
dict_character = {ch:index for index, ch in enumerate(sorted(set(character_list)))}

### Organize data 

In [5]:
seq_len = 40
step = 3

X_list = []
Y_list = []
for start_index in list(range(0,len(long_text) - 100 ,step)):
    X_list.append(long_text[start_index:start_index + seq_len])
    Y_list.append(long_text[start_index + seq_len])

In [29]:
sample_index = 100
print (X_list[sample_index])
print (Y_list[sample_index])

 you added because it seemed kind of spa
m


# Task3 : apply dictionary on charecters to get numbers for each character

In [8]:
X_list_tokenized = []
for line in X_list:
    cur_line_values = []
    for ch in line:
        cur_line_values.append(dict_character[ch])
    X_list_tokenized.append(cur_line_values)

In [9]:
Y_list_tokenized = []
for ch in Y_list:
    Y_list_tokenized.append(dict_character[ch])

In [10]:
from keras.utils import np_utils

Using TensorFlow backend.


In [11]:
X_list_one_hot = []
for line in X_list_tokenized:
    one_hot_line = np_utils.to_categorical(line,num_classes=len(dict_character))
    X_list_one_hot.append(one_hot_line)


In [12]:
Y_list_one_hot = []
for ch in Y_list_tokenized:
    Y_list_one_hot.append(np_utils.to_categorical(ch,num_classes=len(dict_character)))

In [13]:
print(X_list[0][0])
print(X_list_tokenized[0][0])
print(X_list_one_hot[0][0])


N
37
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]


# TASK: define and train a model

In [14]:
import numpy as np

In [15]:
X = np.array(X_list_one_hot)
Y = np.array(Y_list_one_hot)

In [16]:
from keras import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop



In [17]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(seq_len, len(dict_character))))
model.add(Dense(len(dict_character)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [18]:
model.fit(X, Y, epochs=20)

Epoch 1/20
3300/3300 [==============================] - 5s 2ms/step - loss: 3.0422
Epoch 2/20
3300/3300 [==============================] - 5s 2ms/step - loss: 2.5890
Epoch 3/20
3300/3300 [==============================] - 4s 1ms/step - loss: 2.3771
Epoch 4/20
3300/3300 [==============================] - 5s 1ms/step - loss: 2.1354
Epoch 5/20
3300/3300 [==============================] - 5s 1ms/step - loss: 1.9225
Epoch 6/20
3300/3300 [==============================] - 5s 1ms/step - loss: 1.7007
Epoch 7/20
3300/3300 [==============================] - 5s 2ms/step - loss: 1.4371
Epoch 8/20
3300/3300 [==============================] - 5s 2ms/step - loss: 1.2351
Epoch 9/20
3300/3300 [==============================] - 5s 2ms/step - loss: 1.0382
Epoch 10/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.8866
Epoch 11/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.7522
Epoch 12/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.6483
E

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               105472    
_________________________________________________________________
dense_1 (Dense)              (None, 77)                9933      
_________________________________________________________________
activation_1 (Activation)    (None, 77)                0         
Total params: 115,405
Trainable params: 115,405
Non-trainable params: 0
_________________________________________________________________


In [20]:
sentence = long_text[400:440]
sentence_tokenize = [dict_character[ch] for ch in sentence]
new_generated_sentence = []
for i in range(100):
    sentence_tokenize_one_hot = [np_utils.to_categorical(ch,num_classes=len(dict_character)) for ch in sentence_tokenize]
    sentence_tokenize_one_hot = np.array(sentence_tokenize_one_hot)
    next_charecter_prob_vector = model.predict(sentence_tokenize_one_hot[None])
    next_charecter_prob_vector = next_charecter_prob_vector.ravel()
    most_probably_character_index = np.argmax(next_charecter_prob_vector)
    
    sentence_tokenize[:-1] = sentence_tokenize[1:]
    sentence_tokenize[-1] = most_probably_character_index
    
    new_generated_sentence.append(most_probably_character_index)



In [21]:
new_generated_sentence

[47,
 65,
 1,
 47,
 60,
 50,
 1,
 50,
 51,
 51,
 60,
 66,
 1,
 61,
 66,
 51,
 1,
 66,
 61,
 1,
 66,
 54,
 51,
 1,
 49,
 61,
 59,
 62,
 65,
 62,
 51,
 47,
 64,
 51,
 1,
 66,
 54,
 51,
 1,
 64,
 51,
 47,
 50,
 51,
 64,
 65,
 1,
 66,
 61,
 1,
 48,
 67,
 66,
 1,
 47,
 66,
 1,
 55,
 65,
 1,
 47,
 64,
 1,
 47,
 64,
 51,
 1,
 64,
 51,
 47,
 50,
 51,
 64,
 47,
 58,
 1,
 66,
 54,
 51,
 1,
 47,
 58,
 50,
 55,
 49,
 58,
 51,
 1,
 66,
 61,
 1,
 66,
 54,
 51,
 1,
 51,
 70,
 62,
 58,
 51]

In [22]:
tokens_dict = {value:key for key,value in dict_character.items() }

In [23]:
new_generated_sentence_in_english = [tokens_dict[tok] for tok in  new_generated_sentence]

In [24]:
new_line = ''
for c in new_generated_sentence_in_english:1
    new_line += c
print (new_line)

as and deent ote to the compspeare the readers to but at is ar are readeral the aldicle to the exple


In [25]:
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
plt.plot(next_word_prob_vector,'b.')

NameError: name 'next_word_prob_vector' is not defined